In [480]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [495]:
def latent_variables_pc(filepath):
    pca = PCA(n_components=10)
    data = pd.read_csv(filepath,index_col=0)
    data = data.iloc[:, :41]
    data = data.T
    scaler = StandardScaler()
    data_normalized = scaler.fit_transform(data)
    #data_normalized = data_normalized.T
    data_pca = pca.fit_transform(data_normalized)
    return data_pca, pca.explained_variance_ratio_

In [496]:
data_pca_dict = {}
pc_ratio_dict = {}
spike_rate_dir = ['Image_1_sum_spike_rate.csv', 'Image_2_sum_spike_rate.csv', 'Image_3_sum_spike_rate.csv', 'Image_4_sum_spike_rate.csv', 'Image_5_sum_spike_rate.csv', 'Image_6_sum_spike_rate.csv',
                  'Image_7_sum_spike_rate.csv', 'Image_8_sum_spike_rate.csv', 'Image_9_sum_spike_rate.csv', 'Image_10_sum_spike_rate.csv', 'Image_11_sum_spike_rate.csv', 'Image_12_sum_spike_rate.csv',
                  'Image_13_sum_spike_rate.csv', 'Image_14_sum_spike_rate.csv', 'Image_15_sum_spike_rate.csv', 'Image_16_sum_spike_rate.csv', 'Image_17_sum_spike_rate.csv', 'Image_18_sum_spike_rate.csv']

for file in spike_rate_dir:
    data_pca_dict[file], pc_ratio_dict[file] = latent_variables_pc(f'/media/ubuntu/sda/data/mouse6/output_grating/03_single_neuron/Static/sum_spike_rate/{file}')

In [497]:
data_pc1 = pd.DataFrame()
data_pc2 = pd.DataFrame()
data_pc3 = pd.DataFrame()

for file in spike_rate_dir:
    data_pc1 = pd.concat((data_pc1, pd.DataFrame(data_pca_dict[file][:, 0])), axis=1)
    data_pc2 = pd.concat((data_pc2, pd.DataFrame(data_pca_dict[file][:, 1])), axis=1)
    data_pc3 = pd.concat((data_pc3, pd.DataFrame(data_pca_dict[file][:, 2])), axis=1)


In [498]:
data_pc1.to_csv("processed_data/data_pc1_phase1.csv")
data_pc2.to_csv("processed_data/data_pc2_phase1.csv")
data_pc3.to_csv("processed_data/data_pc3_phase1.csv")

In [499]:
import torch.nn.functional as F

with PdfPages("neuron_model/similarity_heatmap_phase1.pdf") as pdf:

    data_pc1.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
    #data_pc1 = data_pc1[[1, 5,6,7,10,11,12,14,17,18, 2,3,4,8,9,13,15,16]]
    data = torch.tensor(data_pc1.T.to_numpy())
    data.unsqueeze_(1) 
    similarity_matrix = F.cosine_similarity(data, data.transpose(0, 1), dim=2)
    sns.heatmap(similarity_matrix.numpy(), cmap="coolwarm", vmin=0, vmax=1)
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5,5))
    sns.heatmap(similarity_matrix.numpy(), cmap="coolwarm", vmin=0, vmax=1, cbar=False, xticklabels=False, yticklabels=False)
    pdf.savefig()
    plt.close()

    data_pc2.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
    data_pc2 = data_pc2[[1,2,3,4,5,6, 9,10,12, 13, 14, 15, 16, 17, 7,8,11,18]]
    data = torch.tensor(data_pc2.T.to_numpy())
    data.unsqueeze_(1) 
    similarity_matrix_2 = F.cosine_similarity(data, data.transpose(0, 1), dim=2)
    sns.heatmap(similarity_matrix_2.numpy(), cmap="coolwarm", vmin=0, vmax=1)
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5,5))
    sns.heatmap(similarity_matrix_2.numpy(), cmap="coolwarm", vmin=0, vmax=1, cbar=False, xticklabels=False, yticklabels=False)
    pdf.savefig()
    plt.close()

    data_pc3.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
    data = torch.tensor(data_pc3.T.to_numpy())
    data.unsqueeze_(1) 
    similarity_matrix_3 = F.cosine_similarity(data, data.transpose(0, 1), dim=2)
    sns.heatmap(similarity_matrix_3.numpy(), cmap="coolwarm", vmin=0, vmax=1)
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5,5))
    sns.heatmap(similarity_matrix_3.numpy(), cmap="coolwarm", vmin=0, vmax=1, cbar=False, xticklabels=False, yticklabels=False)
    pdf.savefig()
    plt.close()

In [500]:
with PdfPages("neuron_model/waveform_phase1.pdf") as pdf:
    plt.figure(figsize=(4, 3))
    for file in spike_rate_dir:
        sns.lineplot(data_pca_dict[file][:, 0])
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(4, 3))
    for file in spike_rate_dir:
        sns.lineplot(data_pca_dict[file][:, 1])
    pdf.savefig()
    plt.close()


    plt.figure(figsize=(4, 3))
    for file in spike_rate_dir:
        sns.lineplot(data_pca_dict[file][:, 2])
    pdf.savefig()
    plt.close()


In [501]:
pc_ratio = pd.DataFrame()

for key ,item in pc_ratio_dict.items():
    pc_ratio = pd.concat((pc_ratio, pd.DataFrame(item)), axis=1)

pc_ratio = pc_ratio.mean(axis=1)
cumulative_sums = []
current_sum = 0
for value in pc_ratio:
    current_sum += value
    cumulative_sums.append(current_sum)

In [502]:
pd.DataFrame(cumulative_sums).to_csv("processed_data/pc_ratio_phase1.csv")

In [470]:
with PdfPages('neuron_model/pc_ration_phase1.pdf') as pdf:
    plt.figure(figsize=(3,2))
    sns.lineplot(x = range(3),
                 y = pc_ratio,
                 color = 'black')
    plt.ylim(0, 1)
    pdf.savefig()
    plt.close()

In [ ]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()

colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)', 'rgb(44, 160, 44)', 'rgb(214, 39, 40)', 
          'rgb(148, 103, 189)', 'rgb(140, 86, 75)', 'rgb(227, 119, 194)', 'rgb(127, 127, 127)', 
          'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

for i, (key, matrix) in enumerate(data_pca_dict.items()):
    color = colors[i % len(colors)]
    
    x = matrix[:, 0]
    y = matrix[:, 1]
    z = matrix[:, 2]
    
    fig.add_trace(go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='lines',
        line=dict(color=color, width=4),
        name=key
    ))

fig.update_layout(
    title='3D Line Plots from Dictionary',
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis',
        xaxis=dict(showgrid=False, gridcolor='white', backgroundcolor='white'),
        yaxis=dict(showgrid=False, gridcolor='white', backgroundcolor='white'),
        zaxis=dict(showgrid=False, gridcolor='white', backgroundcolor='white'),
        bgcolor='white' 
    )
)


fig.show()

- Phase II

In [489]:
def latent_variables_pc(filepath):
    pca = PCA(n_components=10)
    data = pd.read_csv(filepath,index_col=0)
    data = data.iloc[:, 41:]
    data = data.T
    scaler = StandardScaler()
    data_normalized = scaler.fit_transform(data)
    #data_normalized = data_normalized.T
    data_pca = pca.fit_transform(data_normalized)
    return data_pca, pca.explained_variance_ratio_

In [490]:
data_pca_dict = {}
pc_ratio_dict = {}
spike_rate_dir = ['Image_1_sum_spike_rate.csv', 'Image_2_sum_spike_rate.csv', 'Image_3_sum_spike_rate.csv', 'Image_4_sum_spike_rate.csv', 'Image_5_sum_spike_rate.csv', 'Image_6_sum_spike_rate.csv',
                  'Image_7_sum_spike_rate.csv', 'Image_8_sum_spike_rate.csv', 'Image_9_sum_spike_rate.csv', 'Image_10_sum_spike_rate.csv', 'Image_11_sum_spike_rate.csv', 'Image_12_sum_spike_rate.csv',
                  'Image_13_sum_spike_rate.csv', 'Image_14_sum_spike_rate.csv', 'Image_15_sum_spike_rate.csv', 'Image_16_sum_spike_rate.csv', 'Image_17_sum_spike_rate.csv', 'Image_18_sum_spike_rate.csv']

for file in spike_rate_dir:
    data_pca_dict[file], pc_ratio_dict[file] = latent_variables_pc(f'/media/ubuntu/sda/data/mouse6/output_grating/03_single_neuron/Static/sum_spike_rate/{file}')

In [491]:
data_pc1 = pd.DataFrame()
data_pc2 = pd.DataFrame()
data_pc3 = pd.DataFrame()

for file in spike_rate_dir:
    data_pc1 = pd.concat((data_pc1, pd.DataFrame(data_pca_dict[file][:, 0])), axis=1)
    data_pc2 = pd.concat((data_pc2, pd.DataFrame(data_pca_dict[file][:, 1])), axis=1)
    data_pc3 = pd.concat((data_pc3, pd.DataFrame(data_pca_dict[file][:, 2])), axis=1)

In [492]:
data_pc1.to_csv("processed_data/data_pc1_phase2.csv")
data_pc2.to_csv("processed_data/data_pc2_phase2.csv")
data_pc3.to_csv("processed_data/data_pc3_phase2.csv")

In [474]:
import torch.nn.functional as F

with PdfPages("neuron_model/similarity_heatmap_phase2.pdf") as pdf:

    data_pc1.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
    #data_pc1 = data_pc1[[1, 5,6,7,10,11,12,14,17,18, 2,3,4,8,9,13,15,16]]
    data = torch.tensor(data_pc1.T.to_numpy())
    data.unsqueeze_(1) 
    similarity_matrix = F.cosine_similarity(data, data.transpose(0, 1), dim=2)
    sns.heatmap(similarity_matrix.numpy(), cmap="coolwarm", vmin=0, vmax=1)
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5,5))
    sns.heatmap(similarity_matrix.numpy(), cmap="coolwarm", vmin=0, vmax=1, cbar=False, xticklabels=False, yticklabels=False)
    pdf.savefig()
    plt.close()

    data_pc2.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
    data_pc2 = data_pc2[[1,2,3,4,5,6, 9,10,12, 13, 14, 15, 16, 17, 7,8,11,18]]
    data = torch.tensor(data_pc2.T.to_numpy())
    data.unsqueeze_(1) 
    similarity_matrix_2 = F.cosine_similarity(data, data.transpose(0, 1), dim=2)
    sns.heatmap(similarity_matrix_2.numpy(), cmap="coolwarm", vmin=0, vmax=1)
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5,5))
    sns.heatmap(similarity_matrix_2.numpy(), cmap="coolwarm", vmin=0, vmax=1, cbar=False, xticklabels=False, yticklabels=False)
    pdf.savefig()
    plt.close()

    data_pc3.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
    data = torch.tensor(data_pc3.T.to_numpy())
    data.unsqueeze_(1) 
    similarity_matrix_3 = F.cosine_similarity(data, data.transpose(0, 1), dim=2)
    sns.heatmap(similarity_matrix_3.numpy(), cmap="coolwarm", vmin=0, vmax=1)
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(5,5))
    sns.heatmap(similarity_matrix_3.numpy(), cmap="coolwarm", vmin=0, vmax=1, cbar=False, xticklabels=False, yticklabels=False)
    pdf.savefig()
    plt.close()

In [475]:
with PdfPages("neuron_model/waveform_phase2.pdf") as pdf:
    plt.figure(figsize=(4, 3))
    for file in spike_rate_dir:
        sns.lineplot(data_pca_dict[file][:, 0])
    pdf.savefig()
    plt.close()

    plt.figure(figsize=(4, 3))
    for file in spike_rate_dir:
        sns.lineplot(data_pca_dict[file][:, 1])
    pdf.savefig()
    plt.close()


    plt.figure(figsize=(4, 3))
    for file in spike_rate_dir:
        sns.lineplot(data_pca_dict[file][:, 2])
    pdf.savefig()
    plt.close()

In [493]:
pc_ratio = pd.DataFrame()

for key ,item in pc_ratio_dict.items():
    pc_ratio = pd.concat((pc_ratio, pd.DataFrame(item)), axis=1)

pc_ratio = pc_ratio.mean(axis=1)
cumulative_sums = []
current_sum = 0
for value in pc_ratio:
    current_sum += value
    cumulative_sums.append(current_sum)

In [494]:
pd.DataFrame(cumulative_sums).to_csv("processed_data/pc_ratio_phase2.csv")

In [479]:
with PdfPages('neuron_model/pc_ration_phase2.pdf') as pdf:
    plt.figure(figsize=(3,2))
    sns.lineplot(x = range(10),
                 y = cumulative_sums,
                 color = 'black')
    
    plt.ylim(0, 1)
    pdf.savefig()
    plt.close()

In [462]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()

colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)', 'rgb(44, 160, 44)', 'rgb(214, 39, 40)', 
          'rgb(148, 103, 189)', 'rgb(140, 86, 75)', 'rgb(227, 119, 194)', 'rgb(127, 127, 127)', 
          'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

for i, (key, matrix) in enumerate(data_pca_dict.items()):
    color = colors[i % len(colors)]
    
    x = matrix[:, 0]
    y = matrix[:, 1]
    z = matrix[:, 2]
    
    fig.add_trace(go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='lines',
        line=dict(color=color, width=4),
        name=key
    ))

fig.update_layout(
    title='3D Line Plots from Dictionary',
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis',
        xaxis=dict(showgrid=False, gridcolor='white', backgroundcolor='white'),
        yaxis=dict(showgrid=False, gridcolor='white', backgroundcolor='white'),
        zaxis=dict(showgrid=False, gridcolor='white', backgroundcolor='white'),
        bgcolor='white' 
    )
)


fig.show()

In [507]:
pc_ratio_1 = pd.read_csv("/media/ubuntu/sda/data/mouse6/output_grating/03_single_neuron/Static/processed_data/pc_ratio_phase1.csv", index_col=0)
pc_ratio_2 = pd.read_csv("/media/ubuntu/sda/data/mouse6/output_grating/03_single_neuron/Static/processed_data/pc_ratio_phase2.csv", index_col=0)

In [519]:
with PdfPages('neuron_model/pc_ration_phase12.pdf') as pdf:
    plt.figure(figsize=(3,2))
    sns.lineplot(x = range(10),
                 y = pc_ratio_1['0'],
                 color = '#F0868C',
                 linewidth = 3)
    sns.lineplot(x = range(10),
                 y = pc_ratio_2['0'],
                 color = '#70A5D9',
                 linewidth = 3)
    plt.yticks([])
    plt.xticks([])
    plt.ylim(0, 1.1)
    plt.ylabel("")
    pdf.savefig()
    plt.close()

In [524]:
a = np.load("/media/ubuntu/sda/data/mouse6/output/04_get_neuron_channel/segmentation_raw/seg_012123_1_3.npy")

In [525]:
a.shape

(5000, 30)